In [ ]:
from dotenv import load_dotenv
from pymilvus import MilvusClient, DataType
import os

In [ ]:
load_dotenv()
milvus_client = MilvusClient(uri=os.getenv("MILVUS_CONNECTION_URI"))
schema = milvus_client.create_schema()
schema.add_field(field_name="id", datatype=DataType.VARCHAR, is_primary=True, max_length=512, auto_id=True)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=65535)
schema.add_field(field_name="corpus_source_file_name", datatype=DataType.VARCHAR, max_length=127, nullable=True)
schema.add_field(field_name="data_vector", datatype=DataType.FLOAT_VECTOR, dim=768)
schema.add_field(field_name="tags", datatype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=128, max_length=32, nullable=True)
schema.enable_dynamic_field = True

In [ ]:
data_vector_index = milvus_client.prepare_index_params()
data_vector_index.add_index(
    field_name="data_vector",
    index_type="IVF_FLAT",
    index_name="idx_data_vector_ivf_flat",
    metric_type="COSINE"
)

In [ ]:
milvus_client.create_collection(
    collection_name="doc_embeddings",
    schema=schema,
    index_params=data_vector_index,
    enable_dynamic_field=True
)